In [1]:
import importlib
import os
from pathlib import Path
import sys

from arcgis.features import GeoAccessor, GeoSeriesAccessor, FeatureSet
from arcgis.gis import GIS
from dotenv import load_dotenv, find_dotenv
import pandas as pd

In [2]:
# paths to common data locations - NOTE: to convert any path to a raw string, simply use str(path_instance)
dir_prj = Path.cwd().parent.parent

# import the project package from the project package path - only necessary if you are not using a unique environemnt for this project
sys.path.append(str(dir_prj/'src'))
import modeling
from modeling import ModelingAccessor

# load the "autoreload" extension so that code can change, & always reload modules so that as you change code in src, it gets loaded
%load_ext autoreload
%autoreload 2

# load environment variables from .env
load_dotenv(find_dotenv())

True

In [11]:
gis = GIS(
    url=os.getenv('ESRI_PORTAL_URL'), 
    username=os.getenv('ESRI_PORTAL_USERNAME'),
    password=os.getenv('ESRI_PORTAL_PASSWORD')
)
# gis = GIS(
#     url=os.getenv('ESRI_GIS_URL'), 
#     username=os.getenv('ESRI_GIS_USERNAME'),
#     password=os.getenv('ESRI_GIS_PASSWORD')
# )

cntry = modeling.Country('USA', gis)

cntry

<modeling.Country - USA (GIS at https://geoai-ent.bd.esri.com/portal/ logged in as jmccune)>

In [21]:
aoi_df = cntry.cbsas.get('Springfield, MO', return_geometry=False)

aoi_df

,ID,NAME
0,44180,"Springfield, MO Metropolitan Statistical Area"


In [25]:
bg_df = aoi_df.mdl.block_groups.get()

bg_df.head()

,ID,NAME,SHAPE
0,290430201021,290430201.021,"{""rings"": [[[-93.46455800019417, 37.0374819998..."
1,290430201022,290430201.022,"{""rings"": [[[-93.43715000022708, 37.0960800002..."
2,290430202011,290430202.011,"{""rings"": [[[-93.40370449959221, 37.0956520000..."
3,290430202012,290430202.012,"{""rings"": [[[-93.35789799918815, 37.0481250000..."
4,290430202021,290430202.021,"{""rings"": [[[-93.31383149955684, 37.0946480000..."


In [26]:
evars = cntry.enrich_variables

evars.head()

/Users/joel5174/projects/demographic-modeling-module/src/modeling/accessor.py:22: UserWarning: registration of accessor <class 'modeling.accessor.ModelingAccessor'> under name 'mdl' for type <class 'pandas.core.frame.DataFrame'> is overriding a preexisting attribute with the same name.
  @register_dataframe_accessor('mdl')


,name,alias,data_collection,enrich_name,enrich_field_name,description,vintage,units
0,AGE0_CY,2020 Population Age <1,1yearincrements,1yearincrements.AGE0_CY,F1yearincrements_AGE0_CY,2020 Total Population Age <1 (Esri),2020,count
1,AGE1_CY,2020 Population Age 1,1yearincrements,1yearincrements.AGE1_CY,F1yearincrements_AGE1_CY,2020 Total Population Age 1 (Esri),2020,count
2,AGE2_CY,2020 Population Age 2,1yearincrements,1yearincrements.AGE2_CY,F1yearincrements_AGE2_CY,2020 Total Population Age 2 (Esri),2020,count
3,AGE3_CY,2020 Population Age 3,1yearincrements,1yearincrements.AGE3_CY,F1yearincrements_AGE3_CY,2020 Total Population Age 3 (Esri),2020,count
4,AGE4_CY,2020 Population Age 4,1yearincrements,1yearincrements.AGE4_CY,F1yearincrements_AGE4_CY,2020 Total Population Age 4 (Esri),2020,count


In [34]:
e_vars = cntry.enrich_variables
key_vars = e_vars[
    (e_vars.data_collection.str.startswith('Key'))
    & (e_vars.name.str.endswith('CY'))
].reset_index(drop=True)

key_vars

,name,alias,data_collection,enrich_name,enrich_field_name,description,vintage,units
0,TOTPOP_CY,2020 Total Population,KeyUSFacts,KeyUSFacts.TOTPOP_CY,KeyUSFacts_TOTPOP_CY,2020 Total Population (Esri),2020,count
1,GQPOP_CY,2020 Group Quarters Population,KeyUSFacts,KeyUSFacts.GQPOP_CY,KeyUSFacts_GQPOP_CY,2020 Group Quarters Population (Esri),2020,count
2,DIVINDX_CY,2020 Diversity Index,KeyUSFacts,KeyUSFacts.DIVINDX_CY,KeyUSFacts_DIVINDX_CY,2020 Diversity Index (Esri),2020,count
3,TOTHH_CY,2020 Total Households,KeyUSFacts,KeyUSFacts.TOTHH_CY,KeyUSFacts_TOTHH_CY,2020 Total Households (Esri),2020,count
4,AVGHHSZ_CY,2020 Average Household Size,KeyUSFacts,KeyUSFacts.AVGHHSZ_CY,KeyUSFacts_AVGHHSZ_CY,2020 Average Household Size (Esri),2020,count
5,MEDHINC_CY,2020 Median Household Income,KeyUSFacts,KeyUSFacts.MEDHINC_CY,KeyUSFacts_MEDHINC_CY,2020 Median Household Income (Esri),2020,currency
6,AVGHINC_CY,2020 Average Household Income,KeyUSFacts,KeyUSFacts.AVGHINC_CY,KeyUSFacts_AVGHINC_CY,2020 Average Household Income (Esri),2020,currency
7,PCI_CY,2020 Per Capita Income,KeyUSFacts,KeyUSFacts.PCI_CY,KeyUSFacts_PCI_CY,2020 Per Capita Income (Esri),2020,currency
8,TOTHU_CY,2020 Total Housing Units,KeyUSFacts,KeyUSFacts.TOTHU_CY,KeyUSFacts_TOTHU_CY,2020 Total Housing Units (Esri),2020,count
9,OWNER_CY,2020 Owner Occupied HUs,KeyUSFacts,KeyUSFacts.OWNER_CY,KeyUSFacts_OWNER_CY,2020 Owner Occupied Housing Units (Esri),2020,count


In [28]:
key_df = bg_df.mdl.enrich(key_vars)

key_df

,ID,NAME,TOTPOP_CY,GQPOP_CY,DIVINDX_CY,TOTHH_CY,AVGHHSZ_CY,MEDHINC_CY,AVGHINC_CY,PCI_CY,...,VACANT_CY,MEDVAL_CY,AVGVAL_CY,POPGRW10CY,HHGRW10CY,FAMGRW10CY,DPOP_CY,DPOPWRK_CY,DPOPRES_CY,SHAPE
0,290430201021,290430201.021,3379,0,14.7,1277,2.65,49317,56233,21252,...,107,124034,153771,2.08,2.05,1.95,2837,785,2052,"{""rings"": [[[-93.46455800019417, 37.0374819998..."
1,290430201022,290430201.022,2850,0,10.5,1035,2.75,65905,79395,28833,...,51,222645,240043,1.55,1.52,1.44,2032,677,1355,"{""rings"": [[[-93.43715000022708, 37.0960800002..."
2,290430202011,290430202.011,2557,0,15.9,902,2.83,86075,113347,39984,...,55,260032,318187,0.87,0.84,0.77,1791,654,1137,"{""rings"": [[[-93.40370449959221, 37.0956520000..."
3,290430202012,290430202.012,2268,14,13.4,850,2.65,69303,96706,36251,...,38,277902,333359,1.00,0.97,0.87,1798,571,1227,"{""rings"": [[[-93.35789799918815, 37.0481250000..."
4,290430202021,290430202.021,2022,0,20.4,758,2.67,56060,70749,26522,...,71,228096,223473,1.74,1.59,1.47,1956,892,1064,"{""rings"": [[[-93.31383149955684, 37.0946480000..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258,291679601004,291679601.004,825,0,10.6,328,2.52,29263,50035,19893,...,36,96354,202667,0.19,0.12,-0.05,830,281,549,"{""rings"": [[[-93.53509499999579, 37.6763929996..."
259,291679601005,291679601.005,1776,102,9.8,660,2.54,51382,73797,27462,...,70,145577,207341,0.74,0.72,0.58,1584,591,993,"{""rings"": [[[-93.48932999996126, 37.6677680003..."
260,291679601006,291679601.006,657,0,10.7,273,2.41,37718,70120,29137,...,48,116489,130719,0.10,-0.11,-0.28,574,183,391,"{""rings"": [[[-93.42092099947325, 37.6253149999..."
261,292254701011,292254701.011,2515,0,7.9,983,2.56,50137,65708,25682,...,114,169141,191210,1.02,1.06,0.96,1903,457,1446,"{""rings"": [[[-92.84384099989529, 37.4437320000..."


In [23]:
%%time
brnd_df = aoi_df.mdl.business.get_by_name("O'REILLY AUTO PARTS")


brnd_df

CPU times: user 15.6 ms, sys: 1.72 ms, total: 17.4 ms
Wall time: 431 ms


,LOCNUM,CONAME,NAICSDESC,NAICS,SIC,SOURCE,PUBPRV,FRNCOD,ISCODE,CITY,ZIP,STATE,SHAPE,location_id,brand_name,brand_name_category
0,154662258,O'REILLY AUTO PARTS,AUTOMOBILE PARTS & SUPPLIES-RETAIL-NEW,44131011,553111,INFOGROUP,2,6,,BOLIVAR,65613,MO,"{""x"": -93.40667973, ""y"": 37.60543633, ""spatial...",154662258,O'REILLY AUTO PARTS,O'REILLY AUTO PARTS
1,154705453,O'REILLY AUTO PARTS,AUTOMOBILE PARTS & SUPPLIES-RETAIL-NEW,44131011,553111,INFOGROUP,2,6,,MARSHFIELD,65706,MO,"{""x"": -92.9197395, ""y"": 37.3362930000001, ""spa...",154705453,O'REILLY AUTO PARTS,O'REILLY AUTO PARTS
2,154727838,O'REILLY AUTO PARTS,AUTOMOBILE PARTS & SUPPLIES-RETAIL-NEW,44131011,553111,INFOGROUP,2,6,,REPUBLIC,65738,MO,"{""x"": -93.4652303299999, ""y"": 37.12426149, ""sp...",154727838,O'REILLY AUTO PARTS,O'REILLY AUTO PARTS
3,154763866,O'REILLY AUTO PARTS,AUTOMOBILE PARTS & SUPPLIES-RETAIL-NEW,44131011,553111,INFOGROUP,2,6,,SPRINGFIELD,65802,MO,"{""x"": -93.3122069999999, ""y"": 37.2247380000001...",154763866,O'REILLY AUTO PARTS,O'REILLY AUTO PARTS
4,154772479,O'REILLY AUTO PARTS,AUTOMOBILE PARTS & SUPPLIES-RETAIL-NEW,44131011,553111,INFOGROUP,2,6,,SPRINGFIELD,65803,MO,"{""x"": -93.3027165, ""y"": 37.2413700000001, ""spa...",154772479,O'REILLY AUTO PARTS,O'REILLY AUTO PARTS
5,154772487,O'REILLY AUTO PARTS,AUTOMOBILE PARTS & SUPPLIES-RETAIL-NEW,44131011,553111,INFOGROUP,2,6,,SPRINGFIELD,65803,MO,"{""x"": -93.2564159999999, ""y"": 37.2402990000001...",154772487,O'REILLY AUTO PARTS,O'REILLY AUTO PARTS
6,154772495,O'REILLY AUTO PARTS,AUTOMOBILE PARTS & SUPPLIES-RETAIL-NEW,44131011,553111,INFOGROUP,2,6,,SPRINGFIELD,65802,MO,"{""x"": -93.3346215, ""y"": 37.212633, ""spatialRef...",154772495,O'REILLY AUTO PARTS,O'REILLY AUTO PARTS
7,154793541,O'REILLY AUTO PARTS,AUTOMOBILE PARTS & SUPPLIES-RETAIL-NEW,44131011,553111,INFOGROUP,2,6,,SPRINGFIELD,65804,MO,"{""x"": -93.2339655, ""y"": 37.1801790000001, ""spa...",154793541,O'REILLY AUTO PARTS,O'REILLY AUTO PARTS
8,154818769,O'REILLY AUTO PARTS,AUTOMOBILE PARTS & SUPPLIES-RETAIL-NEW,44131011,553111,INFOGROUP,2,6,,SPRINGFIELD,65807,MO,"{""x"": -93.340287, ""y"": 37.183428, ""spatialRefe...",154818769,O'REILLY AUTO PARTS,O'REILLY AUTO PARTS
9,154818777,O'REILLY AUTO PARTS,AUTOMOBILE PARTS & SUPPLIES-RETAIL-NEW,44131011,553111,INFOGROUP,2,6,,SPRINGFIELD,65807,MO,"{""x"": -93.2955075, ""y"": 37.1428020000001, ""spa...",154818777,O'REILLY AUTO PARTS,O'REILLY AUTO PARTS


In [24]:
%%time
comp_df = aoi_df.mdl.business.get_competition(brnd_df, local_threshold=3)

comp_df

CPU times: user 26.6 ms, sys: 2.78 ms, total: 29.3 ms
Wall time: 555 ms


,LOCNUM,CONAME,NAICSDESC,NAICS,SIC,SOURCE,PUBPRV,FRNCOD,ISCODE,CITY,ZIP,STATE,SHAPE,location_id,brand_name,brand_name_category
0,154660229,CHUCK'S AUTO SUPPLY,AUTOMOBILE PARTS & SUPPLIES-RETAIL-NEW,44131011,553111,INFOGROUP,,,,BOLIVAR,65613,MO,"{""x"": -93.4124786899999, ""y"": 37.6129175400001...",154660229,CHUCK'S AUTO SUPPLY,local_brand
1,154704472,J & I AUTOMOTIVE DISTRIBUTORS,AUTOMOBILE PARTS & SUPPLIES-RETAIL-NEW,44131011,553111,INFOGROUP,,,,MARSHFIELD,65706,MO,"{""x"": -92.9078504999999, ""y"": 37.3184910000001...",154704472,J & I AUTOMOTIVE DISTRIBUTORS,local_brand
2,154705891,SINGER AUTO PARTS,AUTOMOBILE PARTS & SUPPLIES-RETAIL-NEW,44131011,553111,INFOGROUP,,3,,MARSHFIELD,65706,MO,"{""x"": -92.922588, ""y"": 37.335987, ""spatialRefe...",154705891,SINGER AUTO PARTS,local_brand
3,154730782,PARTS CITY AUTO PARTS-SEYMOUR,AUTOMOBILE PARTS & SUPPLIES-RETAIL-NEW,44131011,553111,INFOGROUP,,,,SEYMOUR,65746,MO,"{""x"": -92.7686385, ""y"": 37.1459610000001, ""spa...",154730782,PARTS CITY AUTO PARTS-SEYMOUR,local_brand
4,154756092,HTD BATTERIES,AUTOMOBILE PARTS & SUPPLIES-RETAIL-NEW,44131011,553111,INFOGROUP,,,,SPRINGFIELD,65802,MO,"{""x"": -93.2528025, ""y"": 37.210968, ""spatialRef...",154756092,HTD BATTERIES,local_brand
5,154764997,B & N AUTO PARTS,AUTOMOBILE PARTS & SUPPLIES-RETAIL-NEW,44131011,553111,INFOGROUP,,,,SPRINGFIELD,65806,MO,"{""x"": -93.323754, ""y"": 37.2091140000001, ""spat...",154764997,B & N AUTO PARTS,local_brand
6,154768279,H & W AUTO SALVAGE & PARTS,AUTOMOBILE PARTS & SUPPLIES-RETAIL-NEW,44131011,553111,INFOGROUP,,,,SPRINGFIELD,65803,MO,"{""x"": -93.346236, ""y"": 37.232154, ""spatialRefe...",154768279,H & W AUTO SALVAGE & PARTS,local_brand
7,154769392,HEDDEN'S AUTOMOTIVE,AUTOMOBILE PARTS & SUPPLIES-RETAIL-NEW,44131011,553111,INFOGROUP,,,,SPRINGFIELD,65803,MO,"{""x"": -93.3087869999999, ""y"": 37.2372930000001...",154769392,HEDDEN'S AUTOMOTIVE,local_brand
8,201162385,BOHANNON AUTO SVC,AUTOMOBILE PARTS & SUPPLIES-RETAIL-NEW,44131011,553111,INFOGROUP,,,,SPRINGFIELD,65804,MO,"{""x"": -93.2505389999999, ""y"": 37.1888190000001...",201162385,BOHANNON AUTO SVC,local_brand
9,223583790,J & S AUTOMOTIVE,AUTOMOBILE PARTS & SUPPLIES-RETAIL-NEW,44131011,553111,INFOGROUP,,,,SPRINGFIELD,65802,MO,"{""x"": -93.2309235, ""y"": 37.2030300000001, ""spa...",223583790,J & S AUTOMOTIVE,local_brand


In [15]:
brand_repl_lst = [
    (r'TRUE VALUE|TRUE VL', 'TRUE VALUE'),
    (R'MC LENDON|MCLENDON' , 'MCLENDON HARDWARE')
]

for search_re, repl_str in brand_repl_lst:
    
    brand_filter = comp_df.brand_name.str.contains(search_re, regex=True)
    
    comp_df.loc[brand_filter, 'brand_name'] = repl_str
    
comp_df

,LOCNUM,CONAME,NAICSDESC,NAICS,SIC,SOURCE,PUBPRV,FRNCOD,ISCODE,CITY,ZIP,STATE,SHAPE,location_id,brand_name,brand_name_category
0,002890986,MC LENDON HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,,,SUMNER,98390,WA,"{""x"": -122.242365, ""y"": 47.2046040000001, ""spa...",002890986,MCLENDON HARDWARE,local_brand
1,006128854,MCLENDON HARDWARE INC,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,,,RENTON,98057,WA,"{""x"": -122.2140195, ""y"": 47.477943, ""spatialRe...",006128854,MCLENDON HARDWARE,MCLENDON HARDWARE INC
2,174245191,DUVALL TRUE VALUE HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,2,,DUVALL,98019,WA,"{""x"": -121.9853835, ""y"": 47.738907, ""spatialRe...",174245191,TRUE VALUE,local_brand
3,174262691,GATEWAY TRUE VALUE HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,2,,ENUMCLAW,98022,WA,"{""x"": -121.9876155, ""y"": 47.2019940000001, ""sp...",174262691,TRUE VALUE,local_brand
4,174471722,TWEEDY & POPP HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,2,,SEATTLE,98103,WA,"{""x"": -122.3357134, ""y"": 47.6612959300001, ""sp...",174471722,TWEEDY & POPP HARDWARE,local_brand
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,933642043,AIRCRAFT HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,,,EVERETT,98204,WA,"{""x"": -122.2741203, ""y"": 47.90709365, ""spatial...",933642043,AIRCRAFT HARDWARE,local_brand
71,950645150,JENSEN PLUMBING LLC,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,,,SEATTLE,98146,WA,"{""x"": -122.35702, ""y"": 47.4991650000001, ""spat...",950645150,JENSEN PLUMBING LLC,local_brand
72,966020547,EQUIPMENT SALES & SURPLUS,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,,,AUBURN,98001,WA,"{""x"": -122.2504785, ""y"": 47.325294, ""spatialRe...",966020547,EQUIPMENT SALES & SURPLUS,local_brand
73,967153974,MCLENDON HARDWARE INC,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,,,KENT,98031,WA,"{""x"": -122.200308, ""y"": 47.3884200000001, ""spa...",967153974,MCLENDON HARDWARE,MCLENDON HARDWARE INC


In [16]:
comp_df.mdl.business.calculate_brand_name_category(3, inplace=True)

comp_df

,LOCNUM,CONAME,NAICSDESC,NAICS,SIC,SOURCE,PUBPRV,FRNCOD,ISCODE,CITY,ZIP,STATE,SHAPE,location_id,brand_name,brand_name_category
0,002890986,MC LENDON HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,,,SUMNER,98390,WA,"{""x"": -122.242365, ""y"": 47.2046040000001, ""spa...",002890986,MCLENDON HARDWARE,MCLENDON HARDWARE
1,006128854,MCLENDON HARDWARE INC,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,,,RENTON,98057,WA,"{""x"": -122.2140195, ""y"": 47.477943, ""spatialRe...",006128854,MCLENDON HARDWARE,MCLENDON HARDWARE
2,174245191,DUVALL TRUE VALUE HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,2,,DUVALL,98019,WA,"{""x"": -121.9853835, ""y"": 47.738907, ""spatialRe...",174245191,TRUE VALUE,TRUE VALUE
3,174262691,GATEWAY TRUE VALUE HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,2,,ENUMCLAW,98022,WA,"{""x"": -121.9876155, ""y"": 47.2019940000001, ""sp...",174262691,TRUE VALUE,TRUE VALUE
4,174471722,TWEEDY & POPP HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,2,,SEATTLE,98103,WA,"{""x"": -122.3357134, ""y"": 47.6612959300001, ""sp...",174471722,TWEEDY & POPP HARDWARE,local_brand
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,933642043,AIRCRAFT HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,,,EVERETT,98204,WA,"{""x"": -122.2741203, ""y"": 47.90709365, ""spatial...",933642043,AIRCRAFT HARDWARE,local_brand
71,950645150,JENSEN PLUMBING LLC,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,,,SEATTLE,98146,WA,"{""x"": -122.35702, ""y"": 47.4991650000001, ""spat...",950645150,JENSEN PLUMBING LLC,local_brand
72,966020547,EQUIPMENT SALES & SURPLUS,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,,,AUBURN,98001,WA,"{""x"": -122.2504785, ""y"": 47.325294, ""spatialRe...",966020547,EQUIPMENT SALES & SURPLUS,local_brand
73,967153974,MCLENDON HARDWARE INC,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,,,KENT,98031,WA,"{""x"": -122.200308, ""y"": 47.3884200000001, ""spa...",967153974,MCLENDON HARDWARE,MCLENDON HARDWARE


In [18]:
brnd_df['brand_comp'] = 'brand'
comp_df['brand_comp'] = 'competition'

loc_df = pd.concat([brnd_df, comp_df])
loc_df

In [19]:
loc_df.brand_name_category.value_counts().sort_values(ascending=False)

local_brand          58
ACE HARDWARE         37
TRUE VALUE           10
MCLENDON HARDWARE     7
Name: brand_name_category, dtype: int64

In [56]:
from modeling import Country

# create a country object instance
cntry = Country('USA', source=gis)

# create an area of interest
aoi_df = cntry.cbsas.get('Seattle')

# use this area of interest to get brand locations
brand_df = aoi_df.mdl.business.get_by_name('Ace Hardware')

# get competitors and categorize all brands with less than
# three locations as a local brand
comp_df = aoi_df.mdl.business.get_competition(brand_df, local_threshold=3)

# with hardware stores, each True Value has a unique name,
# so it helps to rename these to be correctly recognized
# as a brand of stores
replace_lst = [
    ('TRUE VALUE|TRUE VL', 'TRUE VALUE'),
    ('MC LENDON|MCLENDON', 'MCLENDON HARDWARE')
]
for repl in replace_lst:
    brand_filter = comp_df.brand_name.str.contains(repl[0], regex=True)
    comp_df.loc[brand_filter, 'brand_name'] = repl[1]

# now, with the True Values renamed, we need to recalculate which
# locations are actually local brands
comp_df.mdl.business.calculate_brand_name_category(3, inplace=True)

/Users/joel5174/projects/demographic-modeling-module/src/modeling/accessor.py:22: UserWarning: registration of accessor <class 'modeling.accessor.ModelingAccessor'> under name 'mdl' for type <class 'pandas.core.frame.DataFrame'> is overriding a preexisting attribute with the same name.
  @register_dataframe_accessor('mdl')


In [57]:
from tabulate import tabulate

In [58]:
print(tabulate(comp_df.head(), headers=comp_df.columns, tablefmt='rst'))

====  =========  ===========================  ===============  ========  ======  =========  ========  ========  ========  ========  =====  =======  ==============================================================================  =============  ======================  =====================
  ..     LOCNUM  CONAME                       NAICSDESC           NAICS     SIC  SOURCE     PUBPRV    FRNCOD    ISCODE    CITY        ZIP  STATE    SHAPE                                                                             location_id  brand_name              brand_name_category
====  =========  ===========================  ===============  ========  ======  =========  ========  ========  ========  ========  =====  =======  ==============================================================================  =============  ======================  =====================
   0  002890986  MC LENDON HARDWARE           HARDWARE-RETAIL  44413005  525104  INFOGROUP                                SUMNER    983